In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append("../")

import torch
from transformers import AutoTokenizer
from jiwer import wer
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import TensorBoardLogger

from src.model import chained_seq2seq
from src.data_modules_2 import STTDataModule2

## Instantiate Model Class

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
dm = STTDataModule2()
dm.setup("fit")
dm.setup("test")

train_dataloader = dm.train_dataloader()
val_dataloader = dm.val_dataloader()
test_dataloader = dm.test_dataloader()

In [ ]:
logger = TensorBoardLogger("../model/checkpoints/tb_logs", name="chained_seq2seq", log_graph=False)
model = chained_seq2seq()

In [ ]:
seed_everything(42)
trainer = Trainer(
    max_epochs=5,
    accelerator="auto",
    logger=logger,
    devices=1 if torch.cuda.is_available() else None,
    default_root_dir="../model/checkpoints/"
)

In [ ]:
trainer.fit(model, datamodule=dm, ckpt_path="../model/checkpoints/tb_logs/chained_seq2seq/version_2/checkpoints/epoch=0-step=1815.ckpt")